In [1]:
%matplotlib inline
import os, sys
import numpy as np
from utils_data_exploration import *
import gc

### Directory Structure

In [2]:
%pwd # verify you are in the correct folder

'/home/javier/Documents/YelpRestaurantPhotoClassification/nbs'

In [3]:
dataset_folder = 'places/'

In [4]:
# Assign Path Constants
LESSON_HOME_DIR = os.getcwd()
DATA_DIR = LESSON_HOME_DIR + '/../data/'
TRAIN_PATH = DATA_DIR + '/train_photos/'
VALID_PATH = DATA_DIR + '/valid_photos/'
SHARED_PATH = DATA_DIR + 'shared/'
WEIGHTS_PATH = DATA_DIR + '/weights/'

# Assign Dataset Specific Paths
RESULTS_PATH = DATA_DIR + '/results/' + dataset_folder

FEATURES_PATH = RESULTS_PATH + 'features/'
CLASSIFIERS_PATH = RESULTS_PATH + 'classifiers/'

# Train Set

### Start with the list of business and the FC1 Buisness (see Extract_F1_Features.ipynb)

In [5]:
businesses = np.load(SHARED_PATH+'/businesses_shuffled.npy')
businesses_fc1_features = np.load(FEATURES_PATH + 'businesses_fc1_blueprint.npy')

## 4-fold Cross Validation

In [6]:
nb_folds = 4

folds = dict()
total_restaurants = len(businesses)
restaurants_per_fold = int(len(businesses) / nb_folds)
businesses_idx = np.arange(total_restaurants)

for fold in range(nb_folds):
    folds[fold] = businesses_idx[fold*restaurants_per_fold:(fold+1)*restaurants_per_fold]

### Retrieve the binary labels (apply / does'nt apply) for every attribute

In [7]:
attributes = 9 # possible restaurante attributes
labels_dict = dict()
nb_businesses = len(businesses)

for attribute in range(attributes):
    labels_dict[attribute] = np.zeros(nb_businesses)
    for i, business in enumerate(businesses):
        if attribute in df_train_labels.loc[business]['labels']:
            labels_dict[attribute][i] = 1

In [8]:
labels_dict

{0: array([ 1.,  0.,  1., ...,  1.,  0.,  0.]),
 1: array([ 1.,  1.,  0., ...,  0.,  1.,  1.]),
 2: array([ 0.,  1.,  1., ...,  0.,  1.,  0.]),
 3: array([ 1.,  1.,  0., ...,  1.,  1.,  0.]),
 4: array([ 0.,  1.,  0., ...,  0.,  0.,  0.]),
 5: array([ 1.,  1.,  0., ...,  0.,  1.,  0.]),
 6: array([ 1.,  1.,  1., ...,  0.,  1.,  0.]),
 7: array([ 0.,  1.,  0., ...,  0.,  0.,  0.]),
 8: array([ 0.,  0.,  1., ...,  1.,  0.,  1.])}

# Train classifiers on top of the blueprint bottleneck features per restaurant

In [9]:
from keras import backend as K
from keras import applications
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

Using TensorFlow backend.


In [10]:
K.set_image_dim_ordering('th')

### Create a copy of the VGG16 classifier(image net) -> adding dropout and binarizing the output

In [11]:
def get_binary_vgg16_classifier(drop_rate = 0.5):
    
    # Replicate the fc classifier of vgg net
    model = Sequential()
    model.add(Dropout(drop_rate, input_shape=(4096,)))
    model.add(Dense(4096, activation='relu', name='fc7'))
    model.add(Dropout(drop_rate))
    model.add(Dense(365, activation='relu', name='fc8a'))
    
    # Load pre trained weights for places
    model.load_weights(WEIGHTS_PATH+'places/VGG16_weights.h5', by_name=True)
    
    # Remove the predictions layer and add a binary apply/doesen't apply neuron
    model.pop()
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(lr=5e-06), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# Train 4 classifiers per label, one for each fold

In [12]:
import gc

In [38]:
for attribute in range(attributes):
    for val_fold in range(len(folds)):
        
        K.clear_session()
        gc.collect()
        
        # grab a binary classifier model with imagenet weights
        classifier_model = get_binary_vgg16_classifier()
        
        # grab the training-validation data
        train_idx = []
        train_idx = [fold_idx for i, fold_idx in folds.items() if i != val_fold]
        train_idx = np.array([idx for sublist in train_idx for idx in sublist], dtype=np.int32)
        valid_idx = np.array(folds[val_fold], dtype=np.int32)
        X_train = businesses_fc1_features[train_idx, :]; y_train = labels_dict[attribute][train_idx]
        X_valid = businesses_fc1_features[valid_idx, :]; y_valid = labels_dict[attribute][valid_idx]
        
        # Train the prediction layer (100 epochs) without training fc2 weigths
        for layer in classifier_model.layers:
            if layer.name == 'fc7':
                layer.trainable = False
        
        h = classifier_model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=100)
        np.save(RESULTS_PATH+'/history_prediction_attribute_%d_fold_%d'%(attribute, val_fold), h.history)
        
        # Now Train FC2 -> Run for 150 epochs, adam(lr=1e-06)
        for layer in classifier_model.layers:
            if layer.name == 'fc7':
                layer.trainable = True
        
        classifier_model.compile(optimizer=Adam(lr=5e-06), loss='binary_crossentropy', metrics=['accuracy'])
        h = classifier_model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=100)
        np.save(RESULTS_PATH+'/history_fc2_attribute_%d_fold_%d'%(attribute, val_fold), h.history)
        
        classifier_model.save_weights(RESULTS_PATH+'/ft_200_attribute_%d_fold_%d.h5' % (attribute, val_fold))
        
        

Train on 1497 samples, validate on 499 samples
Epoch 1/100
1497/1497 [==============================] - 1s - loss: 0.6318 - acc: 0.6446 - val_loss: 0.5393 - val_acc: 0.7154
Epoch 2/100
1497/1497 [==============================] - 0s - loss: 0.5550 - acc: 0.7101 - val_loss: 0.4957 - val_acc: 0.7495
Epoch 3/100
1497/1497 [==============================] - 0s - loss: 0.5334 - acc: 0.7268 - val_loss: 0.4728 - val_acc: 0.7876
Epoch 4/100
1497/1497 [==============================] - 0s - loss: 0.5095 - acc: 0.7475 - val_loss: 0.4639 - val_acc: 0.7916
Epoch 5/100
1497/1497 [==============================] - 0s - loss: 0.4878 - acc: 0.7562 - val_loss: 0.4488 - val_acc: 0.8056
Epoch 6/100
1497/1497 [==============================] - 0s - loss: 0.4893 - acc: 0.7508 - val_loss: 0.4428 - val_acc: 0.8016
Epoch 7/100
1497/1497 [==============================] - 0s - loss: 0.4696 - acc: 0.7829 - val_loss: 0.4392 - val_acc: 0.7936
Epoch 8/100
1497/1497 [==============================] - 0s - loss: 0.4

1497/1497 [==============================] - 0s - loss: 0.2347 - acc: 0.9132 - val_loss: 0.3925 - val_acc: 0.8156
Epoch 30/100
1497/1497 [==============================] - 0s - loss: 0.2448 - acc: 0.9031 - val_loss: 0.3929 - val_acc: 0.8176
Epoch 31/100
1497/1497 [==============================] - 0s - loss: 0.2351 - acc: 0.9132 - val_loss: 0.3944 - val_acc: 0.8196
Epoch 32/100
1497/1497 [==============================] - 0s - loss: 0.2263 - acc: 0.9178 - val_loss: 0.3952 - val_acc: 0.8196
Epoch 33/100
1497/1497 [==============================] - 0s - loss: 0.2379 - acc: 0.9165 - val_loss: 0.3939 - val_acc: 0.8196
Epoch 34/100
1497/1497 [==============================] - 0s - loss: 0.2481 - acc: 0.9065 - val_loss: 0.3942 - val_acc: 0.8196
Epoch 35/100
1497/1497 [==============================] - 0s - loss: 0.2342 - acc: 0.9165 - val_loss: 0.3989 - val_acc: 0.8136
Epoch 36/100
1497/1497 [==============================] - 0s - loss: 0.2348 - acc: 0.9218 - val_loss: 0.3968 - val_acc: 0.81

1497/1497 [==============================] - 0s - loss: 0.3260 - acc: 0.8657 - val_loss: 0.4585 - val_acc: 0.7816
Epoch 58/100
1497/1497 [==============================] - 0s - loss: 0.3097 - acc: 0.8758 - val_loss: 0.4620 - val_acc: 0.7836
Epoch 59/100
1497/1497 [==============================] - 0s - loss: 0.3252 - acc: 0.8611 - val_loss: 0.4604 - val_acc: 0.7836
Epoch 60/100
1497/1497 [==============================] - 0s - loss: 0.3126 - acc: 0.8778 - val_loss: 0.4621 - val_acc: 0.7776
Epoch 61/100
1497/1497 [==============================] - 0s - loss: 0.3190 - acc: 0.8798 - val_loss: 0.4665 - val_acc: 0.7836
Epoch 62/100
1497/1497 [==============================] - 0s - loss: 0.3084 - acc: 0.8764 - val_loss: 0.4677 - val_acc: 0.7816
Epoch 63/100
1497/1497 [==============================] - 0s - loss: 0.3069 - acc: 0.8818 - val_loss: 0.4682 - val_acc: 0.7796
Epoch 64/100
1497/1497 [==============================] - 0s - loss: 0.3091 - acc: 0.8758 - val_loss: 0.4608 - val_acc: 0.77

1497/1497 [==============================] - 0s - loss: 0.1994 - acc: 0.9292 - val_loss: 0.4812 - val_acc: 0.7936
Epoch 86/100
1497/1497 [==============================] - 0s - loss: 0.1935 - acc: 0.9279 - val_loss: 0.4799 - val_acc: 0.7836
Epoch 87/100
1497/1497 [==============================] - 0s - loss: 0.1940 - acc: 0.9279 - val_loss: 0.4830 - val_acc: 0.7956
Epoch 88/100
1497/1497 [==============================] - 0s - loss: 0.1886 - acc: 0.9359 - val_loss: 0.4823 - val_acc: 0.7876
Epoch 89/100
1497/1497 [==============================] - 0s - loss: 0.2054 - acc: 0.9225 - val_loss: 0.4802 - val_acc: 0.7896
Epoch 90/100
1497/1497 [==============================] - 0s - loss: 0.1902 - acc: 0.9339 - val_loss: 0.4810 - val_acc: 0.7836
Epoch 91/100
1497/1497 [==============================] - 0s - loss: 0.1834 - acc: 0.9305 - val_loss: 0.4825 - val_acc: 0.8016
Epoch 92/100
1497/1497 [==============================] - 0s - loss: 0.1758 - acc: 0.9405 - val_loss: 0.4903 - val_acc: 0.79

1497/1497 [==============================] - 0s - loss: 0.2500 - acc: 0.9051 - val_loss: 0.4714 - val_acc: 0.8096
Epoch 14/100
1497/1497 [==============================] - 0s - loss: 0.2425 - acc: 0.9071 - val_loss: 0.4662 - val_acc: 0.8156
Epoch 15/100
1497/1497 [==============================] - 0s - loss: 0.2404 - acc: 0.9051 - val_loss: 0.4624 - val_acc: 0.8116
Epoch 16/100
1497/1497 [==============================] - 0s - loss: 0.2397 - acc: 0.9045 - val_loss: 0.4725 - val_acc: 0.8096
Epoch 17/100
1497/1497 [==============================] - 0s - loss: 0.2453 - acc: 0.9058 - val_loss: 0.4775 - val_acc: 0.8096
Epoch 18/100
1497/1497 [==============================] - 0s - loss: 0.2313 - acc: 0.9158 - val_loss: 0.4749 - val_acc: 0.8096
Epoch 19/100
1497/1497 [==============================] - 0s - loss: 0.2403 - acc: 0.9165 - val_loss: 0.4645 - val_acc: 0.8196
Epoch 20/100
1497/1497 [==============================] - 0s - loss: 0.2335 - acc: 0.9051 - val_loss: 0.4691 - val_acc: 0.81

...

1497/1497 [==============================] - 0s - loss: 0.2328 - acc: 0.9138 - val_loss: 0.3517 - val_acc: 0.8517
Epoch 58/100
1497/1497 [==============================] - 0s - loss: 0.2440 - acc: 0.8998 - val_loss: 0.3522 - val_acc: 0.8517
Epoch 59/100
1497/1497 [==============================] - 0s - loss: 0.2413 - acc: 0.9078 - val_loss: 0.3531 - val_acc: 0.8517
Epoch 60/100
1497/1497 [==============================] - 0s - loss: 0.2329 - acc: 0.9078 - val_loss: 0.3524 - val_acc: 0.8537
Epoch 61/100
1497/1497 [==============================] - 0s - loss: 0.2345 - acc: 0.9065 - val_loss: 0.3519 - val_acc: 0.8497
Epoch 62/100
1497/1497 [==============================] - 0s - loss: 0.2294 - acc: 0.9058 - val_loss: 0.3508 - val_acc: 0.8517
Epoch 63/100
1497/1497 [==============================] - 0s - loss: 0.2344 - acc: 0.9045 - val_loss: 0.3545 - val_acc: 0.8617
Epoch 64/100
1497/1497 [==============================] - 0s - loss: 0.2309 - acc: 0.9085 - val_loss: 0.3521 - val_acc: 0.85

1497/1497 [==============================] - 0s - loss: 0.1435 - acc: 0.9492 - val_loss: 0.3733 - val_acc: 0.8637
Epoch 86/100
1497/1497 [==============================] - 0s - loss: 0.1475 - acc: 0.9506 - val_loss: 0.3772 - val_acc: 0.8537
Epoch 87/100
1497/1497 [==============================] - 0s - loss: 0.1365 - acc: 0.9492 - val_loss: 0.3730 - val_acc: 0.8557
Epoch 88/100
1497/1497 [==============================] - 0s - loss: 0.1483 - acc: 0.9472 - val_loss: 0.3769 - val_acc: 0.8577
Epoch 89/100
1497/1497 [==============================] - 0s - loss: 0.1410 - acc: 0.9512 - val_loss: 0.3885 - val_acc: 0.8517
Epoch 90/100
1497/1497 [==============================] - 0s - loss: 0.1450 - acc: 0.9452 - val_loss: 0.3784 - val_acc: 0.8577
Epoch 91/100
1497/1497 [==============================] - 0s - loss: 0.1402 - acc: 0.9426 - val_loss: 0.3737 - val_acc: 0.8577
Epoch 92/100
1497/1497 [==============================] - 0s - loss: 0.1439 - acc: 0.9452 - val_loss: 0.3733 - val_acc: 0.85

1497/1497 [==============================] - 0s - loss: 0.1799 - acc: 0.9319 - val_loss: 0.3383 - val_acc: 0.8697
Epoch 14/100
1497/1497 [==============================] - 0s - loss: 0.1829 - acc: 0.9285 - val_loss: 0.3382 - val_acc: 0.8717
Epoch 15/100
1497/1497 [==============================] - 0s - loss: 0.1806 - acc: 0.9279 - val_loss: 0.3349 - val_acc: 0.8737
Epoch 16/100
1497/1497 [==============================] - 0s - loss: 0.1747 - acc: 0.9446 - val_loss: 0.3345 - val_acc: 0.8737
Epoch 17/100
1497/1497 [==============================] - 0s - loss: 0.1690 - acc: 0.9339 - val_loss: 0.3406 - val_acc: 0.8697
Epoch 18/100
1497/1497 [==============================] - 0s - loss: 0.1851 - acc: 0.9332 - val_loss: 0.3389 - val_acc: 0.8758
Epoch 19/100
1497/1497 [==============================] - 0s - loss: 0.1769 - acc: 0.9319 - val_loss: 0.3378 - val_acc: 0.8758
Epoch 20/100
1497/1497 [==============================] - 0s - loss: 0.1714 - acc: 0.9372 - val_loss: 0.3416 - val_acc: 0.86

1497/1497 [==============================] - 0s - loss: 0.2640 - acc: 0.8891 - val_loss: 0.3319 - val_acc: 0.8557
Epoch 42/100
1497/1497 [==============================] - 0s - loss: 0.2578 - acc: 0.8844 - val_loss: 0.3316 - val_acc: 0.8477
Epoch 43/100
1497/1497 [==============================] - 0s - loss: 0.2532 - acc: 0.8991 - val_loss: 0.3318 - val_acc: 0.8597
Epoch 44/100
1497/1497 [==============================] - 0s - loss: 0.2544 - acc: 0.8931 - val_loss: 0.3332 - val_acc: 0.8417
Epoch 45/100
1497/1497 [==============================] - 0s - loss: 0.2513 - acc: 0.8998 - val_loss: 0.3310 - val_acc: 0.8537
Epoch 46/100
1497/1497 [==============================] - 0s - loss: 0.2510 - acc: 0.9011 - val_loss: 0.3317 - val_acc: 0.8597
Epoch 47/100
1497/1497 [==============================] - 0s - loss: 0.2538 - acc: 0.9031 - val_loss: 0.3338 - val_acc: 0.8437
Epoch 48/100
1497/1497 [==============================] - 0s - loss: 0.2360 - acc: 0.8938 - val_loss: 0.3328 - val_acc: 0.84

1497/1497 [==============================] - 0s - loss: 0.1505 - acc: 0.9466 - val_loss: 0.3649 - val_acc: 0.8497
Epoch 70/100
1497/1497 [==============================] - 0s - loss: 0.1404 - acc: 0.9506 - val_loss: 0.3677 - val_acc: 0.8557
Epoch 71/100
1497/1497 [==============================] - 0s - loss: 0.1423 - acc: 0.9459 - val_loss: 0.3710 - val_acc: 0.8337
Epoch 72/100
1497/1497 [==============================] - 0s - loss: 0.1430 - acc: 0.9472 - val_loss: 0.3724 - val_acc: 0.8337
Epoch 73/100
1497/1497 [==============================] - 0s - loss: 0.1508 - acc: 0.9446 - val_loss: 0.3685 - val_acc: 0.8537
Epoch 74/100
1497/1497 [==============================] - 0s - loss: 0.1402 - acc: 0.9539 - val_loss: 0.3767 - val_acc: 0.8377
Epoch 75/100
1497/1497 [==============================] - 0s - loss: 0.1354 - acc: 0.9539 - val_loss: 0.3711 - val_acc: 0.8417
Epoch 76/100
1497/1497 [==============================] - 0s - loss: 0.1478 - acc: 0.9452 - val_loss: 0.3678 - val_acc: 0.84

## Predict Probabilites of each attribute of the given training restaurants

In [13]:
from keras import backend as K

In [14]:
nb_businesses = len(businesses)
nb_attributes = 9

proba_preds = np.empty((nb_businesses, nb_attributes))

In [17]:
for attribute in range(attributes):
    proba_attribute = np.array([]) # initialize attribute probabilities
    for val_fold in range(len(folds)):

        K.clear_session()
        gc.collect()

        # grab the binary model and load the trained weights
        classifier_model = get_binary_vgg16_classifier()
        classifier_model.load_weights(CLASSIFIERS_PATH+'weights/ft_200_attribute_%d_fold_%d.h5' % (attribute, val_fold))

        val_idx = folds[val_fold]
        # grab the businesses_fc1_features as the training set
        X_val = businesses_fc1_features[val_idx]
        
        # predict the probabilty of the attribute for the given classifier 
        predicted_proba = classifier_model.predict_proba(X_val).flatten()
        print(predicted_proba.shape)
        proba_attribute = np.concatenate((proba_attribute, classifier_model.predict_proba(X_val).flatten()))
    
    proba_preds[:, attribute] = (proba_attribute)
np.save(CLASSIFIERS_PATH+'/probas/'+'train_attributes_probas.npy', proba_preds)

 32/499 [>.............................] - ETA: 0s

## Test Set

### Start with the list of business and the FC1 Buisness (see Extract_F1_Features.ipynb)

In [5]:
businesses = np.load(SHARED_PATH+'/test_businesses_shuffled.npy')
businesses_fc1_features = np.load(FEATURES_PATH + 'test_businesses_fc1_blueprint.npy')

## Make predictions blueprint bottleneck features per restaurant

In [6]:
from keras import backend as K
from keras import applications
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam

Using TensorFlow backend.


In [7]:
K.set_image_dim_ordering('th')

### Create a copy of the VGG16 classifier(image net) -> adding dropout and binarizing the output

In [8]:
def get_binary_vgg16_classifier(drop_rate = 0.5):
    
    # Replicate the fc classifier of vgg net
    model = Sequential()
    model.add(Dropout(drop_rate, input_shape=(4096,)))
    model.add(Dense(4096, activation='relu', name='fc7'))
    model.add(Dropout(drop_rate))
    model.add(Dense(365, activation='relu', name='fc8a'))
    
    # Load pre trained weights for places
    model.load_weights(WEIGHTS_PATH+'places/VGG16_weights.h5', by_name=True)
    
    # Remove the predictions layer and add a binary apply/doesen't apply neuron
    model.pop()
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(lr=5e-06), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

## Predict Probabilites of each attribute of the given training restaurants

In [9]:
from keras import backend as K

In [10]:
nb_businesses = len(businesses)
nb_attributes = 9
nb_folds = 4

proba_preds = np.empty((nb_businesses, nb_attributes))

In [11]:
for attribute in range(nb_attributes):
    proba_attribute = np.zeros(nb_businesses) # initialize attribute probabilities
    for val_fold in range(nb_folds):

        K.clear_session()
        gc.collect()

        # grab the binary model and load the trained weights
        classifier_model = get_binary_vgg16_classifier()
        classifier_model.load_weights(CLASSIFIERS_PATH+'weights/ft_200_attribute_%d_fold_%d.h5' % (attribute, val_fold))

        # grab the businesses_fc1_features as the training set
        X_test = businesses_fc1_features
        
        # predict the probabilty of the attribute for the given classifier 
        predicted_proba = classifier_model.predict_proba(X_test).flatten()
        print(predicted_proba.shape)
        proba_attribute += predicted_proba
    
    proba_preds[:, attribute] = (proba_attribute) / 4
np.save(CLASSIFIERS_PATH+'/probas/'+'test_attributes_probas.npy', proba_preds)

 9536/10000 [===========================>..] - ETA: 0s(10000,)
